In [2]:
from fastai.imports import *

In [3]:
df = pd.read_excel('All2021Cohorts_merged_outlier managed_copy_2.xlsx', sheet_name='MergedRTV_Control2021',header=5)

In [4]:
formal_salaried1 = (df['formal_salaried1'] >= 1.0) & (df['formal_salaried1'] <= 11.0)
formal_salaried2 = (df['formal_salaried2'] >= 1.0) & (df['formal_salaried2'] <= 11.0)
formal_salaried3 = (df['formal_salaried3'] >= 1.0) & (df['formal_salaried3'] <= 11.0)
formal_salaried4 = (df['formal_salaried4'] >= 1.0) & (df['formal_salaried4'] <= 11.0)
formal_salaried5 = (df['formal_salaried5'] >= 1.0) & (df['formal_salaried5'] <= 11.0)
formal_salaried6 = (df['formal_salaried6'] >= 1.0) & (df['formal_salaried6'] <= 11.0)

In [5]:
informal_salaried1 = (df['formal_salaried1'] > 11.0)
informal_salaried2 = (df['formal_salaried2'] > 11.0)
informal_salaried3 = (df['formal_salaried3'] > 11.0)
informal_salaried4 = (df['formal_salaried4'] > 11.0)
informal_salaried5 = (df['formal_salaried5'] > 11.0)
informal_salaried6 = (df['formal_salaried6'] > 11.0)

In [6]:
infcount1 = informal_salaried1.sum()
infcount2 = informal_salaried2.sum()
infcount3 = informal_salaried3.sum()
infcount4 = informal_salaried4.sum()
infcount5 = informal_salaried5.sum()
infcount6 = informal_salaried6.sum()

In [7]:
fcount1,fcount2,fcount3,fcount4,fcount5,fcount6

NameError: name 'fcount1' is not defined

In [8]:
infcount1,infcount2,infcount3,infcount4,infcount5,infcount6

(131, 16, 1, 0, 0, 0)

In [9]:
foverall_count = (formal_salaried1 | formal_salaried2 | formal_salaried3 | formal_salaried4 | formal_salaried5 | formal_salaried6).sum()

In [10]:
foverall_count

309

In [11]:
filtered_df = df[formal_salaried1 | formal_salaried2 | formal_salaried3 | formal_salaried4 | formal_salaried5 | formal_salaried6]
filtered_df.shape

(309, 5663)

In [12]:
infiltered_df = df[informal_salaried1 | informal_salaried2 | informal_salaried3 | informal_salaried4 | informal_salaried5 | informal_salaried6]
infiltered_df.shape

(136, 5663)

In [106]:
peer = df[df['Status']=='PEER']
rtv = df[df['Status']=='RTV']

In [118]:
rtv['Criteria_Label'] = np.nan

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_64862/2090392999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtv['Criteria_Label'] = np.nan


In [119]:
rtv['Criteria_Label'].value_counts()

Series([], Name: count, dtype: int64)

In [120]:
formal_indices = filtered_df.index.intersection(rtv.index)

In [121]:
len(formal_indices)

212

In [122]:
informal_indices = infiltered_df.index.intersection(rtv.index)

In [123]:
len(informal_indices)

93

In [124]:
rtv.loc[formal_indices, 'Criteria_Label'] = 'Formal'

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_64862/3418550005.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Formal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  rtv.loc[formal_indices, 'Criteria_Label'] = 'Formal'


In [125]:
informal_only_indices = informal_indices.difference(formal_indices)

In [126]:
rtv.loc[informal_only_indices, 'Criteria_Label'] = 'inFormal'

In [127]:
both_indices = informal_indices.intersection(formal_indices)
rtv.loc[both_indices, 'Criteria_Label'] = 'Both'

In [128]:
rtv['Criteria_Label'].value_counts()

Criteria_Label
Formal      207
inFormal     88
Both          5
Name: count, dtype: int64

In [129]:
rtv['Criteria_Label'].value_counts(normalize=True)

Criteria_Label
Formal      0.690000
inFormal    0.293333
Both        0.016667
Name: proportion, dtype: float64

In [153]:
dummies = pd.get_dummies(rtv['Criteria_Label'], prefix='Criteria')

In [154]:
dummies.head(2)

,Criteria_Both,Criteria_Formal,Criteria_inFormal
0,False,False,False
1,False,False,False


In [155]:
rtv.shape

(3897, 5664)

In [157]:
rtv = pd.concat([rtv, dummies], axis=1)

In [158]:
rtv.Criteria_Both.value_counts()

Criteria_Both
False    3892
True        5
Name: count, dtype: int64

In [159]:
rtv.shape

(3897, 5667)

In [160]:
rtv.groupby('Criteria_Both')['HH Income (USD_Cons_rate)'].mean()

Criteria_Both
False    483.728954
True     864.370484
Name: HH Income (USD_Cons_rate), dtype: float64

In [161]:
rtv.groupby('Criteria_Formal')['HH Income (USD_Cons_rate)'].mean()

Criteria_Formal
False    456.395556
True     980.170726
Name: HH Income (USD_Cons_rate), dtype: float64

In [105]:
rtv.groupby('Criteria_Formal')[[
    'Formal Employment (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
    'HH Income + Consumption + Residues/Day (USD_Cons_rate)'
]].mean()

,Formal Employment (USD_Cons_rate),HH Income (USD_Cons_rate),HH Income + Consumption + Residues (USD_Cons_rate),HH Income/Day (USD_Cons_rate),HH Income + Consumption + Residues/Day (USD_Cons_rate)
Criteria_Formal,,,,,
False,8.789778,456.395556,791.906881,1.250399,2.169608
True,583.507775,980.170726,1277.651198,2.685399,3.500414


In [104]:
rtv.groupby('Criteria_inFormal')[[
    'Formal Employment (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
    'HH Income + Consumption + Residues/Day (USD_Cons_rate)'
]].mean()

,Formal Employment (USD_Cons_rate),HH Income (USD_Cons_rate),HH Income + Consumption + Residues (USD_Cons_rate),HH Income/Day (USD_Cons_rate),HH Income + Consumption + Residues/Day (USD_Cons_rate)
Criteria_inFormal,,,,,
False,32.459527,476.432420,810.860921,1.305294,2.221537
True,336.159712,821.180157,1114.101606,2.249809,3.052333


In [60]:
rtv.groupby('Criteria_inFormal')['HH Income (USD_Cons_rate)'].mean()

Criteria_inFormal
False    476.432420
True     821.180157
Name: HH Income (USD_Cons_rate), dtype: float64

In [131]:
peer['Criteria_Label'] = np.nan

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_64862/490926834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peer['Criteria_Label'] = np.nan


In [132]:
peer['Criteria_Label'].value_counts()

Series([], Name: count, dtype: int64)

In [133]:
formal_indices_p = filtered_df.index.intersection(peer.index)

In [134]:
len(formal_indices_p)

97

In [135]:
informal_indices_p = infiltered_df.index.intersection(peer.index)

In [136]:
len(informal_indices_p)

43

In [137]:
peer.loc[formal_indices_p, 'Criteria_Label'] = 'Formal'

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_64862/334181936.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Formal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  peer.loc[formal_indices_p, 'Criteria_Label'] = 'Formal'


In [138]:
informal_only_indices_p = informal_indices_p.difference(formal_indices_p)

In [139]:
peer.loc[informal_only_indices_p, 'Criteria_Label'] = 'inFormal'

In [140]:
both_indices_p = informal_indices_p.intersection(formal_indices_p)
peer.loc[both_indices_p, 'Criteria_Label'] = 'Both'

In [141]:
peer['Criteria_Label'].value_counts()

Criteria_Label
Formal      95
inFormal    41
Both         2
Name: count, dtype: int64

In [142]:
peer['Criteria_Label'].value_counts(normalize=True)

Criteria_Label
Formal      0.688406
inFormal    0.297101
Both        0.014493
Name: proportion, dtype: float64

In [143]:
rtv.groupby('Criteria_Label')['HH Income (USD_Cons_rate)'].mean()

Criteria_Label
Both        864.370484
Formal      980.170726
inFormal    821.180157
Name: HH Income (USD_Cons_rate), dtype: float64

In [144]:
dummies_p = pd.get_dummies(peer['Criteria_Label'], prefix='Criteria')

In [145]:
dummies_p.head(2)

,Criteria_Both,Criteria_Formal,Criteria_inFormal
3897,False,False,False
3898,False,False,False


In [146]:
peer.shape

(1899, 5664)

In [147]:
peer.shape

(1899, 5664)

In [148]:
peer = pd.concat([peer, dummies_p], axis=1)

In [149]:
peer.Criteria_Both.value_counts()

Criteria_Both
False    1897
True        2
Name: count, dtype: int64

In [150]:
peer.Criteria_Formal.value_counts()

Criteria_Formal
False    1804
True       95
Name: count, dtype: int64

In [151]:
peer.groupby('Criteria_Both')['HH Income (USD_Cons_rate)'].mean()

Criteria_Both
False     280.682954
True     1004.553083
Name: HH Income (USD_Cons_rate), dtype: float64

In [152]:
peer.groupby('Criteria_Formal')['HH Income (USD_Cons_rate)'].mean()

Criteria_Formal
False    260.571776
True     677.823008
Name: HH Income (USD_Cons_rate), dtype: float64

In [81]:
peer.groupby('Criteria_inFormal')['HH Income (USD_Cons_rate)'].mean()

Criteria_inFormal
False    274.800686
True     582.560839
Name: HH Income (USD_Cons_rate), dtype: float64

In [103]:
peer.groupby('Criteria_Formal')[[
    'Formal Employment (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
    'HH Income + Consumption + Residues/Day (USD_Cons_rate)'
]].mean()

,Formal Employment (USD_Cons_rate),HH Income (USD_Cons_rate),HH Income + Consumption + Residues (USD_Cons_rate),HH Income/Day (USD_Cons_rate),HH Income + Consumption + Residues/Day (USD_Cons_rate)
Criteria_Formal,,,,,
False,9.445979,260.571776,465.591246,0.713895,1.275592
True,460.257281,677.823008,891.638694,1.857049,2.442846


In [102]:
peer.groupby('Criteria_inFormal')[[
    'Formal Employment (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
    'HH Income + Consumption + Residues/Day (USD_Cons_rate)'
]].mean()

,Formal Employment (USD_Cons_rate),HH Income (USD_Cons_rate),HH Income + Consumption + Residues (USD_Cons_rate),HH Income/Day (USD_Cons_rate),HH Income + Consumption + Residues/Day (USD_Cons_rate)
Criteria_inFormal,,,,,
False,24.420608,274.800686,480.338583,0.752879,1.315996
True,375.402351,582.560839,784.468229,1.596057,2.149228


### Formal Employment

In [107]:
concat_series = pd.concat([
    rtv['formal_salaried1'], 
    rtv['formal_salaried2'], 
    rtv['formal_salaried3'],
    rtv['formal_salaried4'], 
    rtv['formal_salaried5'], 
    rtv['formal_salaried6']
])

In [108]:
specified_values = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0,9.0, 10.0, 11.0]

formal_subset = concat_series[concat_series.isin(specified_values)]


informal_subset = concat_series[~concat_series.isin(specified_values)]

In [109]:
formal_subset.value_counts(normalize=True)

1.0     0.519313
4.0     0.223176
8.0     0.051502
7.0     0.047210
2.0     0.042918
5.0     0.034335
11.0    0.034335
3.0     0.021459
9.0     0.017167
10.0    0.008584
Name: proportion, dtype: float64

In [110]:
informal_subset.value_counts(normalize=True)

20.0    0.303922
23.0    0.156863
15.0    0.137255
12.0    0.117647
18.0    0.098039
17.0    0.068627
19.0    0.049020
14.0    0.029412
13.0    0.019608
24.0    0.019608
Name: proportion, dtype: float64

In [111]:
peer_concat_series = pd.concat([
    peer['formal_salaried1'], 
    peer['formal_salaried2'], 
    peer['formal_salaried3'],
    peer['formal_salaried4'], 
    peer['formal_salaried5'], 
    peer['formal_salaried6']
])

In [112]:
peer_specified_values = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0,9.0, 10.0, 11.0]

peer_formal_subset = peer_concat_series[peer_concat_series.isin(peer_specified_values)]


peer_informal_subset = peer_concat_series[~peer_concat_series.isin(peer_specified_values)]

In [113]:
peer_formal_subset.value_counts(normalize=True)

1.0     0.518519
4.0     0.101852
8.0     0.074074
3.0     0.074074
11.0    0.064815
2.0     0.064815
7.0     0.027778
9.0     0.027778
5.0     0.027778
10.0    0.018519
Name: proportion, dtype: float64

In [114]:
peer_informal_subset.value_counts(normalize=True)

20.0    0.456522
18.0    0.173913
12.0    0.130435
19.0    0.108696
15.0    0.065217
23.0    0.043478
24.0    0.021739
Name: proportion, dtype: float64

In [162]:
rtv.groupby(['Criteria_Formal', ''])['']

In [165]:
df[['formal_salaried1','formal_salaried2','formal_salaried3','formal_salaried4' ]]

,formal_salaried1,formal_salaried2,formal_salaried3,formal_salaried4
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
5791,NaN,NaN,NaN,NaN
5792,NaN,NaN,NaN,NaN
5793,NaN,NaN,NaN,NaN
5794,NaN,NaN,NaN,NaN


In [171]:
df[['formal_salaried1', 'formal_salaried2', 'formal_salaried3', 'formal_salaried4']].apply(lambda x: ','.join(map(str, x)), axis=1).value_counts()

nan,nan,nan,nan      5358
1.0,nan,nan,nan       128
4.0,nan,nan,nan        58
20.0,nan,nan,nan       35
1.0,1.0,nan,nan        18
8.0,nan,nan,nan        18
23.0,nan,nan,nan       17
18.0,nan,nan,nan       17
2.0,nan,nan,nan        15
7.0,nan,nan,nan        14
15.0,nan,nan,nan       14
12.0,nan,nan,nan       13
11.0,nan,nan,nan       11
19.0,nan,nan,nan       10
3.0,nan,nan,nan         9
5.0,nan,nan,nan         8
17.0,nan,nan,nan        7
9.0,nan,nan,nan         7
20.0,20.0,nan,nan       6
4.0,1.0,nan,nan         3
10.0,nan,nan,nan        3
24.0,nan,nan,nan        3
8.0,1.0,nan,nan         2
1.0,12.0,nan,nan        2
3.0,5.0,nan,nan         1
1.0,11.0,nan,nan        1
13.0,nan,nan,nan        1
20.0,18.0,nan,nan       1
3.0,1.0,nan,nan         1
20.0,1.0,nan,nan        1
1.0,10.0,nan,nan        1
11.0,2.0,20.0,nan       1
11.0,1.0,nan,nan        1
15.0,15.0,nan,nan       1
11.0,12.0,nan,nan       1
23.0,20.0,nan,nan       1
20.0,4.0,nan,nan        1
3.0,2.0,nan,nan         1
13.0,14.0,na

In [168]:
df['new_variable'] = df[['formal_salaried1', 'formal_salaried2', 'formal_salaried3', 'formal_salaried4']].apply(
    lambda x: ','.join(map(str, [val for val in x if not np.isnan(val)])), axis=1
)

In [184]:
df['new_variable']

0        
1        
2        
3        
4        
       ..
5791     
5792     
5793     
5794     
5795     
Name: new_variable, Length: 5796, dtype: object

In [181]:
df['formal_salaried1'].value_counts()

formal_salaried1
1.0     151
4.0      62
20.0     44
8.0      20
23.0     18
18.0     17
15.0     16
2.0      15
7.0      14
11.0     14
12.0     14
3.0      12
19.0     10
5.0       9
9.0       7
17.0      7
24.0      3
10.0      3
13.0      2
Name: count, dtype: int64

In [183]:
df['work_netpay_salaried1'].count()

440

In [175]:
rtv.groupby('formal_salaried1')['work_netpay_salaried1'].mean()

formal_salaried1
1.0     263809.523810
2.0     215555.555556
3.0     276000.000000
4.0     269803.921569
5.0     268333.333333
7.0      77272.727273
8.0     180000.000000
9.0     200000.000000
10.0    120000.000000
11.0    311428.571429
12.0    166250.000000
13.0     80000.000000
15.0    139230.769231
17.0    154285.714286
18.0    150000.000000
19.0    224000.000000
20.0    126923.076923
23.0    192500.000000
24.0    275000.000000
Name: work_netpay_salaried1, dtype: float64

In [306]:
ds_2 = df[(df['formal_salaried1'] == 1.0) | (df['formal_salaried2'] == 1.0) | (df['formal_salaried3'] == 1.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [308]:
ds_2.head()

,formal_salaried1,work_netpay_salaried1,formal_salaried2,work_netpay_salaried2,formal_salaried3,work_netpay_salaried3
189,1.0,100000.0,NaN,NaN,NaN,NaN
218,4.0,150000.0,1.0,100000.0,NaN,NaN
226,1.0,70000.0,NaN,NaN,NaN,NaN
228,1.0,100000.0,NaN,NaN,NaN,NaN
358,1.0,70000.0,NaN,NaN,NaN,NaN


In [311]:
ds_2.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 1.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 1.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 1.0 else 0),
         axis=1)

189     100000.0
218     100000.0
226      70000.0
228     100000.0
358      70000.0
          ...   
5773    500000.0
5778    360000.0
5779    250000.0
5782    350000.0
5790    300000.0
Length: 159, dtype: float64

## Education

In [226]:
adjusted_cpi = 80.66/100
rate = 3600

In [420]:
Education_rtv = rtv[(rtv['formal_salaried1'] == 1.0) | (rtv['formal_salaried2'] == 1.0) | (rtv['formal_salaried3'] == 1.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [421]:
Education_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 1.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 1.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 1.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

64.45194444444445

In [422]:
Education_peer = peer[(peer['formal_salaried1'] == 1.0) | (peer['formal_salaried2'] == 1.0) | (peer['formal_salaried3'] == 1.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [423]:
Education_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 1.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 1.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 1.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

72.41475555555556

## Health

In [424]:
Health_rtv = rtv[(rtv['formal_salaried1'] == 2.0) | (rtv['formal_salaried2'] == 2.0) | (rtv['formal_salaried3'] == 2.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [425]:
Health_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 2.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 2.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 2.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

50.18844444444444

In [426]:
Health_peer = peer[(peer['formal_salaried1'] == 2.0) | (peer['formal_salaried2'] == 2.0) | (peer['formal_salaried3'] == 2.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [427]:
Health_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 2.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 2.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 2.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

45.131190476190476

## Operation

In [428]:
Operation_rtv = rtv[(rtv['formal_salaried1'] == 3.0) | (rtv['formal_salaried2'] == 3.0) | (rtv['formal_salaried3'] == 3.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [429]:
Operation_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 3.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 3.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 3.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

61.839333333333336

In [430]:
Operation_peer = peer[(peer['formal_salaried1'] == 3.0) | (peer['formal_salaried2'] == 3.0) | (peer['formal_salaried3'] == 3.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [431]:
Operation_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 3.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 3.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 3.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

60.77506944444445

## Security

In [432]:
Security_rtv = rtv[(rtv['formal_salaried1'] == 4.0) | (rtv['formal_salaried2'] == 4.0) | (rtv['formal_salaried3'] == 4.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [433]:
Security_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 4.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 4.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 4.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

59.71942307692308

In [434]:
Security_peer = peer[(peer['formal_salaried1'] == 4.0) | (peer['formal_salaried2'] == 4.0) | (peer['formal_salaried3'] == 4.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [435]:
Security_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 4.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 4.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 4.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

50.92171717171716

## Administration

In [436]:
admin_rtv = rtv[(rtv['formal_salaried1'] == 5.0) | (rtv['formal_salaried2'] == 5.0) | (rtv['formal_salaried3'] == 5.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [437]:
admin_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 5.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 5.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 5.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

61.335208333333334

In [438]:
admin_peer = peer[(peer['formal_salaried1'] == 5.0) | (peer['formal_salaried2'] == 5.0) | (peer['formal_salaried3'] == 5.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [439]:
admin_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 5.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 5.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 5.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

58.254444444444445

## Legal

In [440]:
legal_rtv = rtv[(rtv['formal_salaried1'] == 6.0) | (rtv['formal_salaried2'] == 6.0) | (rtv['formal_salaried3'] == 6.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [441]:
legal_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 6.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 6.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 6.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

In [442]:
legal_peer = peer[(peer['formal_salaried1'] == 6.0) | (peer['formal_salaried2'] == 6.0) | (peer['formal_salaried3'] == 6.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [443]:
legal_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 6.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 6.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 6.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

## Politician

In [444]:
politician_rtv = rtv[(rtv['formal_salaried1'] == 7.0) | (rtv['formal_salaried2'] == 7.0) | (rtv['formal_salaried3'] == 7.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [445]:
politician_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 7.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 7.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 7.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

17.31338383838384

In [446]:
politician_peer = peer[(peer['formal_salaried1'] == 7.0) | (peer['formal_salaried2'] == 7.0) | (peer['formal_salaried3'] == 7.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [447]:
politician_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 7.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 7.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 7.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

25.39296296296296

## Religious Institution

In [448]:
religious_rtv = rtv[(rtv['formal_salaried1'] == 8.0) | (rtv['formal_salaried2'] == 8.0) | (rtv['formal_salaried3'] == 8.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [449]:
religious_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 8.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 8.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 8.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

40.33

In [450]:
religious_peer = peer[(peer['formal_salaried1'] == 8.0) | (peer['formal_salaried2'] == 8.0) | (peer['formal_salaried3'] == 8.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [451]:
religious_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 8.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 8.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 8.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

41.170208333333335

## Social Workers

In [452]:
social_rtv = rtv[(rtv['formal_salaried1'] == 9.0) | (rtv['formal_salaried2'] == 9.0) | (rtv['formal_salaried3'] == 9.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [453]:
social_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 9.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 9.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 9.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

44.81111111111111

In [454]:
social_peer = peer[(peer['formal_salaried1'] == 9.0) | (peer['formal_salaried2'] == 9.0) | (peer['formal_salaried3'] == 9.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [455]:
social_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 9.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 9.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 9.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

59.748148148148154

## Manufacturing

In [456]:
manufa_rtv = rtv[(rtv['formal_salaried1'] == 10.0) | (rtv['formal_salaried2'] == 10.0) | (rtv['formal_salaried3'] == 10.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [457]:
manufa_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 10.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 10.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 10.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

58.254444444444445

In [458]:
manufa_peer = peer[(peer['formal_salaried1'] == 10.0) | (peer['formal_salaried2'] == 10.0) | (peer['formal_salaried3'] == 10.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [459]:
manufa_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 10.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 10.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 10.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

48.17194444444444

## Construction

In [460]:
constr_rtv = rtv[(rtv['formal_salaried1'] == 11.0) | (rtv['formal_salaried2'] == 11.0) | (rtv['formal_salaried3'] == 11.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [461]:
constr_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 11.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 11.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 11.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

69.45722222222223

In [463]:
constr_peer = peer[(peer['formal_salaried1'] == 11.0) | (peer['formal_salaried2'] == 11.0) | (peer['formal_salaried3'] == 11.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [464]:
constr_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 11.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 11.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 11.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

57.93436507936508

# Informal Sector

## Shop business

In [465]:
shop_rtv = rtv[(rtv['formal_salaried1'] == 12.0) | (rtv['formal_salaried2'] == 12.0) | (rtv['formal_salaried3'] == 12.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [466]:
shop_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 12.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 12.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 12.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

38.70050505050505

In [467]:
shop_peer = peer[(peer['formal_salaried1'] == 12.0) | (peer['formal_salaried2'] == 12.0) | (peer['formal_salaried3'] == 12.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [468]:
shop_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 12.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 12.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 12.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

38.46287037037037

## Food vendors

In [469]:
food_rtv = rtv[(rtv['formal_salaried1'] == 13.0) | (rtv['formal_salaried2'] == 13.0) | (rtv['formal_salaried3'] == 13.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [470]:
food_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 13.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 13.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 13.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

17.924444444444443

In [471]:
food_peer = peer[(peer['formal_salaried1'] == 13.0) | (peer['formal_salaried2'] == 13.0) | (peer['formal_salaried3'] == 13.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [472]:
food_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 13.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 13.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 13.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

## Street vendors

In [473]:
street_rtv = rtv[(rtv['formal_salaried1'] == 14.0) | (rtv['formal_salaried2'] == 14.0) | (rtv['formal_salaried3'] == 14.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [474]:
street_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 14.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 14.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 14.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

41.07685185185185

In [475]:
street_peer = peer[(peer['formal_salaried1'] == 14.0) | (peer['formal_salaried2'] == 14.0) | (peer['formal_salaried3'] == 14.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [476]:
street_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 14.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 14.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 14.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

## Agriculture Produce store

In [477]:
agric_rtv = rtv[(rtv['formal_salaried1'] == 15.0) | (rtv['formal_salaried2'] == 15.0) | (rtv['formal_salaried3'] == 15.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [478]:
agric_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 15.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 15.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 15.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

32.40188034188034

In [479]:
agric_peer = peer[(peer['formal_salaried1'] == 15.0) | (peer['formal_salaried2'] == 15.0) | (peer['formal_salaried3'] == 15.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [480]:
agric_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 15.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 15.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 15.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

53.77333333333333

## Diary outlet

In [481]:
diary_rtv = rtv[(rtv['formal_salaried1'] == 16.0) | (rtv['formal_salaried2'] == 16.0) | (rtv['formal_salaried3'] == 16.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [379]:
diary_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 16.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 16.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 16.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

In [482]:
diary_peer = peer[(peer['formal_salaried1'] == 16.0) | (peer['formal_salaried2'] == 16.0) | (peer['formal_salaried3'] == 16.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [483]:
diary_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 16.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 16.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 16.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

## Non-registered school

In [484]:
nonreg_rtv = rtv[(rtv['formal_salaried1'] == 17.0) | (rtv['formal_salaried2'] == 17.0) | (rtv['formal_salaried3'] == 17.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [383]:
nonreg_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 17.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 17.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 17.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

34.56857142857143

In [485]:
nonreg_peer = peer[(peer['formal_salaried1'] == 17.0) | (peer['formal_salaried2'] == 17.0) | (peer['formal_salaried3'] == 17.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [486]:
nonreg_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 17.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 17.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 17.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

## Catering

In [487]:
cater_rtv = rtv[(rtv['formal_salaried1'] == 18.0) | (rtv['formal_salaried2'] == 18.0) | (rtv['formal_salaried3'] == 18.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [488]:
cater_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 18.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 18.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 18.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

33.608333333333334

In [489]:
cater_peer = peer[(peer['formal_salaried1'] == 18.0) | (peer['formal_salaried2'] == 18.0) | (peer['formal_salaried3'] == 18.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [490]:
cater_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 18.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 18.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 18.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

25.486319444444444

## Building and Construction

In [396]:
buidandCo_rtv = rtv[(rtv['formal_salaried1'] == 19.0) | (rtv['formal_salaried2'] == 19.0) | (rtv['formal_salaried3'] == 19.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [397]:
buidandCo_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 19.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 19.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 19.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

50.18844444444444

In [394]:
buidandCo_peer = peer[(peer['formal_salaried1'] == 19.0) | (peer['formal_salaried2'] == 19.0) | (peer['formal_salaried3'] == 19.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [395]:
buidandCo_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 19.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 19.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 19.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

64.97611111111111

## House Helper

In [398]:
helper_rtv = rtv[(rtv['formal_salaried1'] == 20.0) | (rtv['formal_salaried2'] == 20.0) | (rtv['formal_salaried3'] == 20.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [399]:
helper_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 20.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 20.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 20.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

38.3245633654321

In [400]:
helper_peer = peer[(peer['formal_salaried1'] == 20.0) | (peer['formal_salaried2'] == 20.0) | (peer['formal_salaried3'] == 20.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [401]:
helper_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 20.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 20.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 20.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

52.71201754385965

## Entertainment/music/dance groups

In [402]:
entert_rtv = rtv[(rtv['formal_salaried1'] == 21.0) | (rtv['formal_salaried2'] == 21.0) | (rtv['formal_salaried3'] == 21.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [406]:
entert_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 21.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 21.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 21.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

In [404]:
entert_peer = peer[(peer['formal_salaried1'] == 21.0) | (peer['formal_salaried2'] == 21.0) | (peer['formal_salaried3'] == 21.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [407]:
entert_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 21.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 21.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 21.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

## Cinema/Videography

In [408]:
cinema_rtv = rtv[(rtv['formal_salaried1'] == 22.0) | (rtv['formal_salaried2'] == 22.0) | (rtv['formal_salaried3'] == 22.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [409]:
cinema_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 22.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 22.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 22.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

In [410]:
cinema_peer = peer[(peer['formal_salaried1'] == 22.0) | (peer['formal_salaried2'] == 22.0) | (peer['formal_salaried3'] == 22.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [411]:
cinema_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 22.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 22.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 22.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

nan

## Salaried cleaners

In [412]:
salaried_rtv = rtv[(rtv['formal_salaried1'] == 23.0) | (rtv['formal_salaried2'] == 23.0) | (rtv['formal_salaried3'] == 23.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [413]:
salaried_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 23.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 23.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 23.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

43.130694444444444

In [414]:
salaried_peer = peer[(peer['formal_salaried1'] == 23.0) | (peer['formal_salaried2'] == 23.0) | (peer['formal_salaried3'] == 23.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [415]:
salaried_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 23.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 23.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 23.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

50.4125

## Fuel station attendant

In [416]:
fuel_rtv = rtv[(rtv['formal_salaried1'] == 24.0) | (rtv['formal_salaried2'] == 24.0) | (rtv['formal_salaried3'] == 24.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [417]:
fuel_rtv.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 24.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 24.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 24.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

61.61527777777778

In [418]:
fuel_peer = peer[(peer['formal_salaried1'] == 24.0) | (peer['formal_salaried2'] == 24.0) | (peer['formal_salaried3'] == 24.0)][['formal_salaried1', 'work_netpay_salaried1', 'formal_salaried2', 'work_netpay_salaried2', 'formal_salaried3', 'work_netpay_salaried3']]

In [419]:
fuel_peer.apply(lambda x: 
         (x['work_netpay_salaried1'] if x['formal_salaried1'] == 24.0 else 0) +
         (x['work_netpay_salaried2'] if x['formal_salaried2'] == 24.0 else 0) +
         (x['work_netpay_salaried3'] if x['formal_salaried3'] == 24.0 else 0),
         axis=1).mean() * adjusted_cpi / rate

45.93138888888889